## **STEP1. 랭체인으로 GPT 부르기**

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from dotenv import load_dotenv
import os

📌 핵심 개념:
- PromptTemplate: 변수를 포함한 프롬프트 정의
- system 메시지: AI의 역할/성격 지정
- human 메시지: 사용자의 실제 입력
- 템플릿을 쓰면 프롬프트 재사용이 쉬워짐

In [2]:
load_dotenv("key.env")

llm=ChatOpenAI(model_name="gpt-4.1-mini", temperature=0)
prompt=ChatPromptTemplate.from_messages([
    ("system", "당신은 친절한 선생님입니다. 초등학생도 이해할 수 있게 설명해주세요."),
    ("user", "{question}"),
])

#prompt와 llm을 파이프로 연결
chain = prompt | llm

#llm 단독호출
response = chain.invoke("양자역학이 뭐야?")
print(response.content)

좋은 질문이야! 양자역학은 아주아주 작은 세상, 예를 들어 원자나 전자처럼 눈에 보이지 않는 작은 입자들이 어떻게 움직이고 행동하는지를 연구하는 과학이야.

우리가 일상에서 보는 큰 물건들은 보통 뉴턴의 법칙으로 잘 설명할 수 있는데, 아주 작은 세계에서는 그 법칙들이 잘 맞지 않아. 그래서 과학자들이 새롭게 만든 법이 바로 양자역학이야.

양자역학에서는 작은 입자들이 마치 파도처럼 움직이기도 하고, 동시에 여러 곳에 있을 수도 있어. 그리고 우리가 그 입자를 관찰하면 그때서야 위치가 확실해진단다.

쉽게 말하면, 양자역학은 아주 작은 세계의 신기한 규칙들을 알려주는 과학이라고 생각하면 돼!


In [3]:
!pip install -U langchain-google-genai

In [4]:
from getpass import getpass

In [5]:
# Gemini API Key
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Gemini API Key: ")

print("✅ OpenAI & Gemini API Key 설정 완료!")

✅ OpenAI & Gemini API Key 설정 완료!


| 패키지                   | 역할                                       |
| --------------------- | ---------------------------------------- |
| langchain-core      | **핵심 추상 개념 (Prompt, Runnable, Message)** |
| langchain           | 체인, 메모리, 에이전트 구현                         |
| langchain-openai    | OpenAI 연동                                |
| langchain-community | 서드파티 도구                                  |

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash"
)

response = llm.invoke("안녕?")
print(response.content)

안녕!


## **Step 2: 메모리 관리 (대화 기록)**

[핵심 개념]
- ChatMessageHistory: 대화 기록을 저장하는 클래스
- RunnableWithMessageHistory: 메모리를 포함한 체인
- session_id: 대화 세션을 구분하는 ID
- store: 여러 세션의 대화 기록을 저장하는 딕셔너리

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

In [8]:
#1. llm
llm = ChatOpenAI(model_name="gpt-4.1-mini")

#2. prompt (history를 끼워 넣는 것이 핵심)
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

#3. 세션별 대화 저장소
store = {}

def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#4. chain
chain = prompt | llm

#5. memory를 체인에 연결
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_variable_name="history"
)

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 이전 대화를 기억하는 AI입니다."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

store = {}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

chain = prompt | llm

chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 세션1: 첫 번째 대화
print("=== 세션1 시작 ===")
config1 = {"configurable": {"session_id": "session1"}}

r1 = chain_with_memory.invoke({"input": "내 이름은 철수야"}, config=config1)
print(f"AI: {r1.content}")

r2 = chain_with_memory.invoke({"input": "내 이름이 뭐였지?"}, config=config1)
print(f"AI: {r2.content}")

# 세션2: 새로운 대화 (이전 내용 기억 못함)
print("\n=== 세션2 시작 (다른 세션) ===")
config2 = {"configurable": {"session_id": "session2"}}

r3 = chain_with_memory.invoke({"input": "내 이름이 뭐였지?"}, config=config2)
print(f"AI: {r3.content}")
print("\n결과: 세션2는 세션1의 대화를 기억하지 못함")

# 다시 세션1로 돌아가면 기억함
print("\n=== 세션1로 복귀 ===")
r4 = chain_with_memory.invoke({"input": "내 이름 다시 말해줘"}, config=config1)
print(f"AI: {r4.content}")
print("\n결과: 세션1은 여전히 '철수'를 기억함")

=== 세션1 시작 ===
AI: 안녕하세요, 철수님! 어떻게 도와드릴까요?
AI: 당신의 이름은 철수입니다. 어떻게 도와드릴까요?

=== 세션2 시작 (다른 세션) ===
AI: 죄송하지만 이전에 당신의 이름을 알려주신 적이 없어서 기억하지 못해요. 이름을 알려주시면 기억할게요!

결과: 세션2는 세션1의 대화를 기억하지 못함

=== 세션1로 복귀 ===
AI: 당신의 이름은 철수입니다.

결과: 세션1은 여전히 '철수'를 기억함


# Step 3: Tool 추가

In [10]:
from langchain_openai import ChatOpenAI
import os

llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 파일 삭제
if os.path.exists("todos.txt"):
    os.remove("todos.txt")

# GPT에게 파일 저장 요청
response = llm.invoke("todos.txt에 '파이썬 공부'를 추가해줘")
print(f"GPT 답변: {response.content}")

# 실제 확인
if os.path.exists("todos.txt"):
    print("파일 있음")
else:
    print("파일 없음 - GPT는 말만 했음")

GPT 답변: `todos.txt` 파일에 '파이썬 공부'를 추가하려면, 다음과 같이 하면 됩니다:

```python
with open('todos.txt', 'a', encoding='utf-8') as file:
    file.write('파이썬 공부\n')
```

이 코드는 `todos.txt` 파일을 열어서(없으면 새로 만들고), 파일 끝에 '파이썬 공부'라는 문장을 한 줄 추가합니다. `encoding='utf-8'`은 한글이 깨지지 않도록 설정한 것입니다.
파일 없음 - GPT는 말만 했음


In [11]:
from langchain_core.tools import tool

In [12]:
# step4_define_tools.py

from langchain_core.tools import tool

@tool
def add_todo(task: str) -> str:
    """할 일을 todos.txt에 추가합니다"""
    with open("todos.txt", "a", encoding="utf-8") as f:
        f.write(f"[ ] {task}\n")
    return f"{task} 추가됨"

@tool
def show_todos() -> str:
    """할 일 목록을 보여줍니다"""
    try:
        with open("todos.txt", "r", encoding="utf-8") as f:
            return f"할 일:\n{f.read()}"
    except:
        return "할 일 없음"

# Tool 확인
print(f"Tool 이름: {add_todo.name}")
print(f"Tool 설명: {add_todo.description}")

# 테스트
print(add_todo.invoke({"task": "파이썬 공부"}))
print(show_todos.invoke({}))


Tool 이름: add_todo
Tool 설명: 할 일을 todos.txt에 추가합니다
파이썬 공부 추가됨
할 일:
[ ] 파이썬 공부



In [13]:
from langchain_core.prompts import PromptTemplate

prompt_template = """
당신은 할 일 관리 AI입니다.

사용 가능한 도구:
{tools}

형식:
Question: {input}
Thought: 생각
Action: [{tool_names}]
Action Input: 입력
Observation: 결과
...
Final Answer: 답변

{agent_scratchpad}
"""

prompt = PromptTemplate.from_template(prompt_template)
print(f"입력 변수: {prompt.input_variables}")

입력 변수: ['agent_scratchpad', 'input', 'tool_names', 'tools']


In [14]:
pip install langgraph

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install -U langchain langchain-community langchain-openai langchain-core langchain-text-splitters

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
# [중요] 최신 LangGraph 방식 import
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv("key.env")

# -------------------------
# 1. LLM 정의
# -------------------------
# [수정] model 파라미터 이름 확인 (gpt-4o 또는 gpt-4.1-mini)
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# -------------------------
# 2. Tools 정의
# -------------------------
@tool
def add_todo(task: str) -> str:
    """할 일 목록 파일(todos.txt)에 새로운 할 일을 추가합니다."""
    with open("todos.txt", "a", encoding="utf-8") as f:
        f.write(f"[ ] {task}\n")
    return f"'{task}' 내용이 추가되었습니다."

@tool
def show_todos() -> str:
    """todos.txt 파일에 저장된 모든 할 일 목록을 읽어서 보여줍니다."""
    try:
        if not os.path.exists("todos.txt"):
            return "할 일 목록이 비어있습니다."
        with open("todos.txt", "r", encoding="utf-8") as f:
            content = f.read()
            return f"현재 할 일 목록:\n{content}" if content else "할 일이 없습니다."
    except Exception as e:
        return f"파일 읽기 에러: {str(e)}"

tools = [add_todo, show_todos]

# -------------------------
# 3. Agent 생성 (LangGraph 방식)
# -------------------------
# [핵심 수정] 
# 1. AgentExecutor를 사용하지 않습니다. (create_react_agent가 실행기 역할까지 함)
# 2. 파라미터 이름: llm -> model
# 3. 시스템 프롬프트: prompt 객체 대신 state_modifier 문자열 사용
agent_app = create_react_agent(
    model=llm,
    tools=tools,
    state_modifier="당신은 할 일 관리 도우미입니다. 사용자의 요청을 정확히 수행하세요."
)

# -------------------------
# 4. 실행
# -------------------------
# [수정] invoke 입력 방식이 변경되었습니다. (input -> messages)
print("--- 에이전트 실행 중... ---")

response = agent_app.invoke({
    "messages": [("human", "파이썬 공부를 할 일에 추가해줘")]
})

# -------------------------
# 5. 결과 출력
# -------------------------
# LangGraph의 결과는 메시지 리스트로 나옵니다. 마지막 메시지가 AI의 답변입니다.
print("\n[최종 답변]")
print(response["messages"][-1].content)

/tmp/ipykernel_32191/2471326151.py:52: LangGraphDeprecatedSinceV10: create_react_agent has been moved to `langchain.agents`. Please update your import to `from langchain.agents import create_agent`. Deprecated in LangGraph V1.0 to be removed in V2.0.
  agent = create_react_agent(


TypeError: create_react_agent() missing 1 required positional argument: 'model'

In [17]:
!pip install -U ddgs

  Using cached lxml-6.0.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl.metadata (3.6 kB)
Using cached lxml-6.0.2-cp312-cp312-manylinux_2_26_x86_64.manylinux_2_28_x86_64.whl (5.3 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 28.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 33.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9/9 [ddgs]5/9 [hpack]
